In [1]:
import sys
sys.path.append("../../../") # Where to find utilssphering
import utilssphering
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

In [2]:
# Read CSV
match_rep_df = pd.read_csv("../../../checkpoints/match_rep_df.csv")
match_rep_df

,Vendor,Batch,Assay_Plate_Barcode,sphering,value_95_replicating,Percent_Replicating,Plate_Map_Name,Modality,Images_per_well,Sites-SubSampled,...,simultaneous-excitation,sites,Size_MB,Size_MB_std,channel_names,brightfield_z_plane_used,feature_channels_found,Percent_Matching,value_95_matching,cell_count
0,MolDev,Scope1_MolDev_10X,Plate2_PCO_6ch_4site_10XPA,True,0.191908,60.000000,JUMP-MOA_compound_platemap,Confocal,4,NaN,...,NaN,4,52.476334,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",23.255814,0.288099,2014937
1,MolDev,Scope1_MolDev_10X,Plate2_PCO_6ch_4site_10XPA,False,0.269727,53.333333,JUMP-MOA_compound_platemap,Confocal,4,NaN,...,NaN,4,52.476334,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.320855,2014937
2,MolDev,Scope1_MolDev_10X,Plate3_PCO_6ch_4site_10XPA_Crest,True,0.269617,62.222222,JUMP-MOA_compound_platemap,Confocal,4,NaN,...,NaN,4,52.476514,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.398249,2413350
3,MolDev,Scope1_MolDev_10X,Plate3_PCO_6ch_4site_10XPA_Crest,False,0.329074,45.555556,JUMP-MOA_compound_platemap,Confocal,4,NaN,...,NaN,4,52.476514,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",18.604651,0.389170,2413350
4,MolDev,Scope1_MolDev_10X_4siteZ,Plate3_PCO_6ch_4site_10XPA_Crestz,True,0.205121,66.666667,JUMP-MOA_compound_platemap,Confocal,4,NaN,...,NaN,4,52.476631,0.000142,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA",23.255814,0.363114,2381443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,BRO0117059_20X,False,0.628379,20.000000,JUMP-MOA_compound_platemap,Confocal,9,4.0,...,4.0,4,7.954439,0.000583,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",6.976744,0.772160,489099
358,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,BRO01177034_20x,True,0.139090,56.666667,JUMP-MOA_compound_platemap,Confocal,9,4.0,...,1.0,4,8.003584,0.000014,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",18.604651,0.193171,452567
359,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,BRO01177034_20x,False,0.666818,21.111111,JUMP-MOA_compound_platemap,Confocal,9,4.0,...,1.0,4,8.003584,0.000014,"AGP, BrightField, DNA, ER, Mito, RNA",Z17,"AGP, BrightField, DNA, ER, Mito, RNA",9.302326,0.790813,452567
360,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,True,0.193787,53.333333,JUMP-MOA_compound_platemap,Confocal,9,4.0,...,4.0,4,7.971628,0.080201,"AGP, BrightField, DNA, ER, Mito, RNA",Z08,"AGP, BrightField, DNA, ER, Mito, RNA",20.930233,0.234702,116595


# Microscopes used by each vendor

- Yokogawa US
  - Yokogawa CV8000
- Yokogawa Japan
  - CV7000 (unsure that this is right. Need confirmation)
- Perkin Elmer
  - Opera Phenix Plus
- Molecular Devices
  - ImageXpress Micro Confocal (IXM-Confocal)
- Nikon
  - Eclipse Ti2 inverted microscope

In [3]:
moldev_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    'spinning-disc',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("MolDev")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[moldev_cols]
df["Place"] = df.reset_index(drop=True).index+1
# Move place column to first position
df.insert(0, "Place", df.pop("Place"))
df
# df.to_csv("(A)-moldev_leaderboard.csv", index=False)

,Place,Percent_Score,Vendor,Modality,Binning,Magnification,aperture,Number_of_channels,z_plane,spinning-disc,channel_names,sites
8,1,100.0,MolDev,Confocal,1,20,0.75,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",9
4,2,98.8,MolDev,Confocal,1,10,0.45,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
0,3,91.5,MolDev,Confocal,1,10,0.45,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",4
2,4,88.8,MolDev,Confocal,1,10,0.45,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
10,5,87.7,MolDev,Confocal,1,20,0.75,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",9
6,6,83.9,MolDev,Confocal,1,20,0.75,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
5,7,80.4,MolDev,Confocal,1,10,0.45,6,3,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
1,8,79.0,MolDev,Confocal,1,10,0.45,6,1,no,"Actin, DNA, ER, Golgi, Mito, RNA",4
3,9,70.5,MolDev,Confocal,1,10,0.45,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",4
11,10,70.3,MolDev,Confocal,1,20,0.75,6,1,yes,"Actin, DNA, ER, Golgi, Mito, RNA",9


In [4]:
yokogawa_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    "simultaneous-excitation",
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Yokogawa")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[yokogawa_cols]
df

# df.to_csv("(B)-yokogawa_leaderboard.csv", index=False)

,Percent_Score,Vendor,Modality,Binning,Magnification,aperture,Number_of_channels,simultaneous-excitation,z_plane,dry-immersion,Percent_Score,channel_names,sites
100,100.0,Yokogawa_US,Confocal,1,20,1.00,5,2.0,12,water,100.0,"AGP, DNA, ER, Mito, RNA",9
90,97.5,Yokogawa_Japan,Confocal,1,20,0.75,5,NaN,10,dry,97.5,"AGP, DNA, ER, Mito, RNA",9
98,93.9,Yokogawa_US,Confocal,1,20,1.00,5,2.0,1,water,93.9,"AGP, DNA, ER, Mito, RNA",9
102,93.8,Yokogawa_US,Confocal,1,20,1.00,6,4.0,12,water,93.8,"AGP, BrightField, DNA, ER, Mito, RNA",9
94,88.9,Yokogawa_US,Confocal,1,10,0.40,6,NaN,12,dry,88.9,"AGP, BrightField, DNA, ER, Mito, RNA",4
96,88.6,Yokogawa_US,Confocal,1,20,1.00,5,2.0,1,water,88.6,"AGP, DNA, ER, Mito, RNA",9
101,87.6,Yokogawa_US,Confocal,1,20,1.00,5,2.0,12,water,87.6,"AGP, DNA, ER, Mito, RNA",9
103,86.3,Yokogawa_US,Confocal,1,20,1.00,6,4.0,12,water,86.3,"AGP, BrightField, DNA, ER, Mito, RNA",9
104,86.2,Yokogawa_US,Confocal,1,20,1.00,6,1.0,12,water,86.2,"AGP, BrightField, DNA, ER, Mito, RNA",9
99,83.9,Yokogawa_US,Confocal,1,20,1.00,5,2.0,1,water,83.9,"AGP, DNA, ER, Mito, RNA",9


In [5]:
nikon_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("Nikon")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[nikon_cols]

df.to_csv("(C)-nikon_leaderboard.csv", index=False)

In [6]:
pe_cols = [
    'Percent_Score',
    'Vendor',
    'Modality',
    'Binning',
    'Magnification',
    'aperture',
    'Number_of_channels',
    'z_plane',
    "dry-immersion",
    'Percent_Score',
    # 'Percent_Replicating',
    # 'Percent_Matching',
    'channel_names',
    'sites'
]

df = match_rep_df[
    (match_rep_df["Vendor"].str.contains("PE")) &
    (~match_rep_df["Batch"].str.contains("siteSub"))
].copy()

# Create an aggregation score for replicating/matching and normalize to the max value
df["Percent_Score"] = df[["Percent_Replicating", "Percent_Matching"]].mean(axis=1)
df["Percent_Score"] = (df["Percent_Score"] / df["Percent_Score"].max()) * 100
df = df.round({"Percent_Score": 1})

df = df.sort_values("Percent_Score", ascending=False)[pe_cols]

df.to_csv("(D)-pe_leaderboard.csv", index=False)